<a href="https://colab.research.google.com/github/hscrown/madatpublicdata/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모듈과 패키지 인스톨과 임포트


In [1]:
# !pip install surprise

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise.model_selection import GridSearchCV

# 데이터로드 및 전처리

In [3]:
df = pd.read_csv('/content/user_place_data_240429.csv')

In [4]:
# 정수인코딩하기 # 도서관은 0 , 박물관은 1, 공원은 2 로 매핑
muse = pd.read_csv('/content/seoul_museums.csv')
muse = muse[['시설명','위도','경도']]

parks = pd.read_csv('/content/seoul_parks.csv')
parks = parks[['공원명','Y좌표(WGS84)','X좌표(WGS84)']]
# ['시설명','위도','경도']로 컬럼명 변경
parks.columns = ['시설명','위도','경도']

libs = pd.read_csv('/content/seoul_libs.csv')
libs = libs[['도서관명','위도','경도']]
libs.columns = ['시설명','위도','경도']

# df2에 합치기
df2 = pd.concat([muse,parks,libs])

# df2에 Place Type 컬럼 만들고, 도서관은 0, 박물관은 1, 공원은 2로 매핑
df2['Place Type'] = 0
df2.loc[df2['시설명'].isin(muse['시설명']), 'Place Type'] = 1
df2.loc[df2['시설명'].isin(parks['시설명']), 'Place Type'] = 2

# df2 컬럼명 변경
df2.columns = ['Place Name','Latitude','Longitude','Place Type']

In [5]:
# df에 df2 머지 # 이너조인
df = pd.merge(df, df2, on='Place Name' , how='inner')

In [6]:
df

,User Id,Place Id,Rating,Place Name,Latitude,Longitude,Place Type
0,1685,34,1,길동생태공원,37.540394,127.154779,2
1,1166,34,5,길동생태공원,37.540394,127.154779,2
2,936,34,3,길동생태공원,37.540394,127.154779,2
3,82,34,4,길동생태공원,37.540394,127.154779,2
4,757,34,1,길동생태공원,37.540394,127.154779,2
...,...,...,...,...,...,...,...
4475,482,159,1,중랑상봉도서관,37.593096,127.078711,0
4476,717,159,5,중랑상봉도서관,37.593096,127.078711,0
4477,785,159,5,중랑상봉도서관,37.593096,127.078711,0
4478,2978,159,1,중랑상봉도서관,37.593096,127.078711,0


In [7]:
# Surprise 라이브러리를 사용하기 위한 데이터 형태로 변환
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['User Id', 'Place Id', 'Rating']], reader)

# 모델 트레이닝 및 평가
- KNNWithMeans는 각 사용자의 평가 경향을 고려한다. 사용자가 주는 평점의 평균을 계산하고, 이를 각 평점에서 빼주어 "평균 조정 평점"을 사용한다.
- 최근접 이웃 선택: 각 예측을 위해, 알고리즘은 가장 유사한 k개의 이웃을 찾고 이 이웃들의 평점을 이용하여 현재 예측하려는 사용자의 평점을 예측

In [18]:
# 데이터를 트레이닝 셋과 테스트 셋으로 분리
trainset, testset = train_test_split(data, test_size=0.25, random_state=39)

# KNNWithMeans 모델을 사용하여 트레이닝
model = KNNWithMeans(k=3, sim_options={'name': 'pearson_baseline', 'user_based': True})
model.fit(trainset)

# 테스트 데이터로 평가
predictions = model.test(testset)
accuracy.rmse(predictions)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0940


1.094010945729359

In [9]:
# random_state = 42 RMSE: 1.1496 1.1495527230403806
# random_state=41 RMSE: 1.1386 1.1385928254113178
# 40 , RMSE: 1.2199 1.2199035609764548
# 39, RMSE: 1.0940, 1.094010945729359
# 38, RMSE: 1.2187 1.2186563641857953
# 37, RMSE: 1.1506 1.1505571265027348

In [19]:
# Suprise라이브러리의 cross-validation 기능이용해서 최적의 k값찾기
# k 값을 다양하게 설정하고 교차 검증 수행
# Results for k=3: Mean RMSE: 1.147337273012189, Mean MAE: 0.7694120979476271
for k in range(1,50):
    model = KNNWithMeans(k=k, sim_options={'name': 'pearson_baseline', 'user_based': True})
    cv_results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    print(f"Results for k={k}: Mean RMSE: {cv_results['test_rmse'].mean()}, Mean MAE: {cv_results['test_mae'].mean()}")
    # rmse가 최소일때 cv_results출력
    if cv_results['test_rmse'].mean() == min(cv_results['test_rmse']):
        print(f"Best k: {k}")


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2208  1.1960  1.1071  1.1790  1.1212  1.1648  0.0437  
MAE (testset)     0.8123  0.7981  0.7519  0.7897  0.7422  0.7788  0.0271  
Fit time          0.40    0.37    0.37    0.39    0.38    0.38    0.01    
Test time         0.02    0.02    0.02    0.02    0.

# 특정 사용자에게 추천 생성

In [21]:
# 특정 사용자에 대한 추천 생성
user_id = 33
user_items = set([j for (j, _) in trainset.ur[trainset.to_inner_uid(user_id)]])
all_items = set(trainset.all_items())
non_rated_items = all_items - user_items
test_items = [[trainset.to_raw_uid(user_id), trainset.to_raw_iid(item), 5.] for item in non_rated_items]
predictions = model.test(test_items)
top_n_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:5]

# 추천 결과 출력
# print(top_n_recommendations)

# 추천 결과 포매팅 및 출력
formatted_results = []

for prediction in top_n_recommendations:
    formatted_result = f"Place ID: {prediction.iid}, Estimated Rating: {prediction.est}"
    formatted_results.append(formatted_result)

# 포매팅된 결과를 한 줄에 하나씩 출력
for result in formatted_results:
    print(result)


Place ID: 152, Estimated Rating: 5
Place ID: 166, Estimated Rating: 5
Place ID: 177, Estimated Rating: 5
Place ID: 79, Estimated Rating: 5
Place ID: 67, Estimated Rating: 5
